In [1]:
from pyprojroot import here
import sys
# spyder up to find the root
root = here(project_files=[".here"])
# append to path
sys.path.append(str(here()))

In [2]:
import geemap.eefolium as geemap
from src.data import ee_download

def generate_polygon(bbox):
    """
    Generates a list of coordinates: [[x1,y1],[x2,y2],[x3,y3],[x4,y4],[x1,y1]]
    """
    return [[bbox[0],bbox[1]],
             [bbox[2],bbox[1]],
             [bbox[2],bbox[3]],
             [bbox[0],bbox[3]],
             [bbox[0],bbox[1]]]

In [3]:
import ee

ee.Initialize()
from datetime import datetime, timedelta

date_event = datetime.strptime("19-11-2020","%d-%m-%Y")
lat,lng = 15.445372970198596, -87.91212639442612


date_end_search = date_event + timedelta(days=20)
lng_size, lat_size = .1,.1
bbox = (lng-lng_size,lat-lat_size, 
        lng+lng_size,lat+lat_size)

loc = ee.Geometry.Polygon(generate_polygon(bbox))

img_col,n_images_col = ee_download.get_collection("COPERNICUS/S2",date_event,date_end_search,loc)

assert n_images_col > 0, "No images were found for those conditions"
    

img_col = ee_download.collection_mosaic_day(img_col, loc)
n_images_col = int(img_col.size().getInfo())
print(f"There are {n_images_col} images in the collection")

Map = geemap.Map()

imgs_list = img_col.toList(n_images_col, 0)
for i in range(n_images_col):
    img_show = ee.Image(imgs_list.get(i))
    Map.addLayer(img_show.clip(loc), 
                 {"min":0, "max":3000, "bands":["B4","B3","B2"]},f"S2 {i}", True)

Map.centerObject(loc)
Map.addLayerControl()
Map

There are 4 images in the collection


In [4]:
import geopandas as gpd
feat_col = ee_download.img_collection_to_feature_collection(img_col)
filename = ee_download.export_eeFeatureCollection(feat_col)
data = gpd.read_file(filename)
data["date"] = data["system:time_start"].apply(lambda x: datetime.utcfromtimestamp(x/1000.))
data

,id,system:time_start,geometry,date
0,2020-11-22,1606024298910,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",2020-11-22 05:51:38.910
1,2020-11-27,1606456298910,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",2020-11-27 05:51:38.910
2,2020-12-02,1606888298910,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",2020-12-02 05:51:38.910
3,2020-12-07,1607320298910,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",2020-12-07 05:51:38.910


In [6]:
# Export 
BANDS_EXPORT = ["B1", "B2", "B3", "B4", "B5",
                "B6", "B7", "B8", "B8A", "B9",
                "B10", "B11", "B12", "QA60"]

img_export = ee.Image(imgs_list.get(1))
img_export = img_export.select(BANDS_EXPORT).resample('bicubic').clip(loc)

task = ee.batch.Export.image.toDrive(img_export,
                                     folder="ee_ipl_uv_downloads",
                                     description="S2_worldfloods",
                                     crs='EPSG:4326', # Use CRS of image to mask methane
                                     scale=10,
                                     formatOptions={"cloudOptimized": True}) # To display nicely in GEE

task.start()

In [7]:
task.status()

{'state': 'READY',
 'description': 'S2_worldfloods',
 'creation_timestamp_ms': 1614072396541,
 'update_timestamp_ms': 1614072396541,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'MLGNV5LE3VZT45FXFISKWGWI',
 'name': 'projects/earthengine-legacy/operations/MLGNV5LE3VZT45FXFISKWGWI'}